In [ ]:
import ccxt
import pandas as pd
from paper_trading_bot1 import Strategy


symbol = "TON/USDT"
timeframe = "1h"
limit = 1000

initial_balance = 1000
risk_per_trade = 0.01
RR_list = [1, 2, 3, 4, 5]


#______FETCH DATA_________

exchange = ccxt.bitget({"options": {"defaultType": "future"}})
ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit = limit)

df = pd.DataFrame(ohlcv,columns=["timestamp", "open", "high", "low", "close", "volume"]
)


#_____BACKTEST FUNCTION________

def backtest_rr(rr):
    strategy = Strategy()
    balance = initial_balance
    position = None
    entry = sl = tp = trades = wins = 0


    for i in range(len(df)):
        data = df.iloc[: i + 1]
        candle = data.iloc[-1]
        signal = strategy.generate_signal(data)

        # ENTRY
        if signal == "BUY" and not position:
            entry = candle.close
            risk_amount = balance * risk_per_trade

            sl = entry * (1 - risk_per_trade)
            tp = entry * (1 + risk_per_trade * rr)

            position = True
            continue

        # EXIT
        if position:
            if candle.low <= sl:
                balance -= risk_amount
                trades += 1
                position = False

            elif candle.high >= tp:
                balance += risk_amount * rr
                trades += 1
                wins += 1
                position = False

    winrate = (wins / trades * 100) if trades else 0

    return {
        "RR": f"1:{rr}",
        "Trades": trades,
        "Win Rate (%)": round(winrate, 2),
        "Final Balance": round(balance, 2)
    }


#__________RUN MULTI-RR_____
print("\n📊 MULTI-RR BACKTEST RESULTS\n")

results = []
for rr in RR_list:
    r = backtest_rr(rr)
    results.append(r)

    print(
        f"RR {r['RR']} | "
        f"Trades: {r['Trades']} | "
        f"Win Rate: {r['Win Rate (%)']}% | "
        f"Balance: {r['Final Balance']}"
    )

print("\nSUMMARY TABLE\n")
print(pd.DataFrame(results))


📊 MULTI-RR BACKTEST RESULTS

RR 1:1 | Trades: 160 | Win Rate: 53.12% | Balance: 1096.37
RR 1:2 | Trades: 105 | Win Rate: 34.29% | Balance: 1019.61
RR 1:3 | Trades: 82 | Win Rate: 25.61% | Balance: 1007.69
RR 1:4 | Trades: 67 | Win Rate: 19.4% | Balance: 967.68
RR 1:5 | Trades: 49 | Win Rate: 16.33% | Balance: 978.49

SUMMARY TABLE

    RR  Trades  Win Rate (%)  Final Balance
0  1:1     160         53.12        1096.37
1  1:2     105         34.29        1019.61
2  1:3      82         25.61        1007.69
3  1:4      67         19.40         967.68
4  1:5      49         16.33         978.49
